In [ ]:
import tensorflow as tf
import os
import subprocess
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
DATASET_DIR = "dataset_processed"

IMG_SIZE = (400, 400)
BATCH_SIZE = 4
EPOCHS = 200
LEARNING_RATE = 1e-4


In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3]
)

train_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"
)

train_gen.class_indices


In [ ]:
print("📁 Dataset info")
print("-" * 40)

print(f"Total imagens (train): {train_gen.n}")
print(f"Total imagens (val):   {val_gen.n}")
print()

print("Classes e índices:")
for cls, idx in train_gen.class_indices.items():
    print(f"  {cls}: {idx}")

print()

# Contagem por classe (treino)
import numpy as np
unique, counts = np.unique(train_gen.classes, return_counts=True)

print("Distribuição no TREINO:")
for cls, idx in train_gen.class_indices.items():
    print(f"  {cls}: {counts[idx]} imagens")


In [ ]:
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)


In [ ]:
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


early_stop = EarlyStopping(
    monitor="val_loss",
    patience=20,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=20,
    min_lr=1e-6
)

callbacks = [early_stop, reduce_lr]


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)


In [ ]:
# Criar pasta output se não existir
os.makedirs("output", exist_ok=True)


# Salvar modelo Keras
model_path = "output/amora_model.h5"
model.save(model_path)

print("Modelo salvo em:", model_path)

# Converter automaticamente para TensorFlow.js
tfjs_output_path = "output/amora_model_tfjs"

subprocess.run([
    "tensorflowjs_converter",
    "--input_format=keras",
    model_path,
    tfjs_output_path
], check=True)

print("Modelo convertido para TensorFlow.js em:", tfjs_output_path)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

